In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install contractions
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import contractions
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from datasets import Dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DSF Project/one-million-reddit-jokes.csv')



In [ ]:
df = df[(df['selftext'] != '[removed]')]
df = df[(df['selftext'] != '[deleted]')]
df.drop(columns=['domain', 'url'], inplace=True)

# Deleting the rows with null selftext as the number of rows is very less
df.dropna(subset=['selftext'], inplace=True)
df = df[df['selftext'] != '']

# Deleting the rows with null title as the number of rows is very less
df.dropna(subset=['title'], inplace=True)
df = df[df['title'] != '']

df.head()

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,selftext,title,score
0,post,ftbp1i,2qh72,jokes,False,1585785543,https://old.reddit.com/r/Jokes/comments/ftbp1i...,My corona is covered with foreskin so it is no...,I am soooo glad I'm not circumcised!,2
1,post,ftboup,2qh72,jokes,False,1585785522,https://old.reddit.com/r/Jokes/comments/ftboup...,It's called Google Sheets.,Did you know Google now has a platform for rec...,9
2,post,ftbopj,2qh72,jokes,False,1585785508,https://old.reddit.com/r/Jokes/comments/ftbopj...,The vacuum doesn't snore after sex.\n\n&amp;#x...,What is the difference between my wife and my ...,15
7,post,ftbeb4,2qh72,jokes,False,1585784461,https://old.reddit.com/r/Jokes/comments/ftbeb4...,Oo..lala...,What did the French man say to the attractive ...,2
10,post,ftbahg,2qh72,jokes,False,1585784081,https://old.reddit.com/r/Jokes/comments/ftbahg...,"Yo momma's so fat, that when she went to the z...",Yo Mama,0


In [ ]:
df['X'] = df['title'] + ' ' + df['selftext']
df.head()


,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,selftext,title,score,X
0,post,ftbp1i,2qh72,jokes,False,1585785543,https://old.reddit.com/r/Jokes/comments/ftbp1i...,My corona is covered with foreskin so it is no...,I am soooo glad I'm not circumcised!,2,I am soooo glad I'm not circumcised! My corona...
1,post,ftboup,2qh72,jokes,False,1585785522,https://old.reddit.com/r/Jokes/comments/ftboup...,It's called Google Sheets.,Did you know Google now has a platform for rec...,9,Did you know Google now has a platform for rec...
2,post,ftbopj,2qh72,jokes,False,1585785508,https://old.reddit.com/r/Jokes/comments/ftbopj...,The vacuum doesn't snore after sex.\n\n&amp;#x...,What is the difference between my wife and my ...,15,What is the difference between my wife and my ...
7,post,ftbeb4,2qh72,jokes,False,1585784461,https://old.reddit.com/r/Jokes/comments/ftbeb4...,Oo..lala...,What did the French man say to the attractive ...,2,What did the French man say to the attractive ...
10,post,ftbahg,2qh72,jokes,False,1585784081,https://old.reddit.com/r/Jokes/comments/ftbahg...,"Yo momma's so fat, that when she went to the z...",Yo Mama,0,"Yo Mama Yo momma's so fat, that when she went ..."


In [ ]:
new_df = df[['X','score']]
new_df.head()

,X,score
0,I am soooo glad I'm not circumcised! My corona...,2
1,Did you know Google now has a platform for rec...,9
2,What is the difference between my wife and my ...,15
7,What did the French man say to the attractive ...,2
10,"Yo Mama Yo momma's so fat, that when she went ...",0


In [ ]:
new_df = new_df.sample(frac=0.25, random_state=42)
new_df.head()

,X,score
693240,do androids dream of electric sheep? Not unles...,1
745616,"I wanted to escape my country, so I went all t...",0
620542,What's that useless piece of skin around a wom...,0
995074,A good ole boy was driving his pickup truck in...,1
439884,I told a fencing joke on Reddit once. Turned o...,3416


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load your data
texts = new_df['X'].values
scores = new_df['score'].values

# Preprocess text (using Keras Tokenizer)
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=250, padding='post')

# Normalize scores using z-score normalization
scaler = StandardScaler()
scores_scaled = scaler.fit_transform(scores.reshape(-1, 1)).flatten()

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, scores_scaled, test_size=0.2, random_state=42
)

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.long).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device).unsqueeze(1)

# Create DataLoader for batch processing
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Load pre-trained GloVe embeddings
embedding_index = {}
with open("/content/drive/MyDrive/DSF Project/glove.6B/glove.6B.300d.txt", "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Create embedding matrix
embedding_dim = 300
embedding_matrix = np.zeros((10000, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < 10000:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Convert embedding matrix to tensor
embedding_matrix_tensor = torch.tensor(embedding_matrix, dtype=torch.float32)

# Define the model
class LSTMModel(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim=128, output_dim=1):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.lstm = nn.LSTM(embedding_matrix.size(1), hidden_dim, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(hidden_dim * 2, 128)
        self.fc2 = nn.Linear(128, output_dim)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = torch.mean(x, 1)  # Global average pooling
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model
model = LSTMModel(embedding_matrix_tensor).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}")

    # Save the model every 10 epochs
    if (epoch + 1) % 10 == 0:
        model_save_path = f"lstm_model_epoch_{epoch+1}.pth"
        torch.save(model.state_dict(), model_save_path)
        print(f"Model saved at epoch {epoch+1} to {model_save_path}")

# Evaluation
model.eval()
predicted_scores = []
true_scores = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted_scores.append(outputs.cpu().numpy())
        true_scores.append(labels.cpu().numpy())

# Combine predictions and true scores
predicted_scores = np.concatenate(predicted_scores)
true_scores = np.concatenate(true_scores)

# Rescale scores back to the original scale
predicted_scores_rescaled = scaler.inverse_transform(predicted_scores)
true_scores_rescaled = scaler.inverse_transform(true_scores)

# Create DataFrame with results
result_df = pd.DataFrame({
    'Text': [" ".join(tokenizer.sequences_to_texts([seq])) for seq in X_test],
    'True_Score': true_scores_rescaled.flatten(),
    'Predicted_Score': predicted_scores_rescaled.flatten()
})

# Display a sample
print(result_df.head())

# Save results to a CSV file
result_df.to_csv("predicted_scores.csv", index=False)
print("Results saved to predicted_scores.csv")


Using device: cuda
Epoch [1/100], Average Loss: 1.0063
Epoch [2/100], Average Loss: 0.9794
Epoch [3/100], Average Loss: 0.9209
Epoch [4/100], Average Loss: 0.8516
Epoch [5/100], Average Loss: 0.7647
Epoch [6/100], Average Loss: 0.6951
Epoch [7/100], Average Loss: 0.6364
Epoch [8/100], Average Loss: 0.5874
Epoch [9/100], Average Loss: 0.5489
Epoch [10/100], Average Loss: 0.5110
Model saved at epoch 10 to lstm_model_epoch_10.pth
Epoch [11/100], Average Loss: 0.4878
Epoch [12/100], Average Loss: 0.4572
Epoch [13/100], Average Loss: 0.4232
Epoch [14/100], Average Loss: 0.4050
Epoch [15/100], Average Loss: 0.3865
Epoch [16/100], Average Loss: 0.3506
Epoch [17/100], Average Loss: 0.3417
Epoch [18/100], Average Loss: 0.3166
Epoch [19/100], Average Loss: 0.2969
Epoch [20/100], Average Loss: 0.2856
Model saved at epoch 20 to lstm_model_epoch_20.pth
Epoch [21/100], Average Loss: 0.2754
Epoch [22/100], Average Loss: 0.2637
Epoch [23/100], Average Loss: 0.2477
Epoch [24/100], Average Loss: 0.2509
